<a href="https://colab.research.google.com/github/Rober157/TFM2/blob/master/ClusterCrossAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R
listOfPackages <- c('data.table','bit64','igraph','dplyr')
for (i in listOfPackages){
     if(! i %in% installed.packages()){
         install.packages(i, dependencies = TRUE)
     }
     require(i)
}

In [ ]:
%%R

library(dplyr)
library(data.table)
library(bit64)
library(igraph)

In [ ]:
%%R

cluster_ratios = data.frame()

for(i in 0:9){
    ratios = c()
    for(j in 0:9){
    if((i == 5) | (j == 5)){
        next}
    if(j <= i){
      ratios = c(ratios,0)
      next}
        name_cross = paste0("./drive/My Drive/TFM/Cluster",i,j,".csv")
        name_1 = paste0("./drive/My Drive/TFM/Cluster",i,".csv")
        name_2 = paste0("./drive/My Drive/TFM/Cluster",j,".csv")
        data_cluster_cross = fread((name_cross))
        data_cluster_1 = fread((name_1))
        data_cluster_2 = fread((name_2))
        network_cross = graph.data.frame(data_cluster_cross)
        network_1 = graph.data.frame(data_cluster_1)
        network_2 = graph.data.frame(data_cluster_2)
        edges = gsize(network_cross)
        possible_edges = gorder(network_1)*gorder(network_2)*2
        ratio = edges/possible_edges
        ratios = c(ratios,ratio)
} 
cluster_ratios = rbind(cluster_ratios,ratios)
}

x = c("Cluster 0", "Cluster 1", "Cluster 2","Cluster 3","Cluster 4","Cluster 6","Cluster 7","Cluster 8","Cluster 9")
colnames(cluster_ratios) = x
rownames(cluster_ratios) = x

In [ ]:
%%R

cluster_ratios[,-1]

In [ ]:
%%R

edges = fread(("./drive/My Drive/TFM/edges_final.csv"))
modus = fread(("./drive/My Drive/TFM/modus.csv"))

In [ ]:
%%R

ratios = matrix(0, nrow = 11, ncol = 11)

for(i in 0:9){
    for(j in 0:9){
    if((i == 5) | (j == 5)){
        next}
    if(j == i){
      next}
ids_1 = modus[modularity_class == i,Id]
ids_2 = modus[modularity_class == j,Id]
indices_from = edges[,from] %in% ids_1
indices_to = edges[,to] %in% ids_2
comb = indices_from & indices_to
new_edges = edges[comb,]
edges_count = nrow(new_edges)
name_1 = paste0("./drive/My Drive/TFM/Cluster",i,".csv")
name_2 = paste0("./drive/My Drive/TFM/Cluster",j,".csv")
data_cluster_1 = fread((name_1))
data_cluster_2 = fread((name_2))
network_1 = graph.data.frame(data_cluster_1)
network_2 = graph.data.frame(data_cluster_2)
possible_edges = gorder(network_1)*gorder(network_2)
ratio = edges_count/possible_edges
ratios[(i+1),(j+1)] = ratio
} 
}

In [ ]:
%%R
write.csv(ratios,'ratios.csv',row.names = FALSE,col.names = FALSE)
